In [14]:
%pip install datasets humanize

Note: you may need to restart the kernel to use updated packages.


In [17]:
import os
import pyarrow as pa
import pandas as pd

PATH = '/fsx/mistobaan/datasets/S2ORC_ver2/'
arrow_file = os.path.join(PATH, 'dataset.arrow')

# pd.read_feather(arrow_file).head()

# Read the Arrow file
# arrow_table = pf.read_table(arrow_file)

# Convert the Arrow table to a Pandas DataFrame
# df = arrow_table.to_pandas()
# Read the Arrow file
arrow_stream = pa.input_stream(arrow_file)
arrow_record_batch = pa.RecordBatchStreamReader(arrow_stream).read_next_batch()

# Convert the Apache Arrow RecordBatch to a Pandas DataFrame
df = pa.Table.from_batches([arrow_record_batch]).to_pandas()
df.iloc[0].text[:20]

'Abstract\nWhereas rad'

In [26]:
import datasets
import os
import humanize

# PATH = '/fsx/home-spirit/s2orc/dataset.arrow'
PATH = '/fsx/mistobaan/datasets/S2ORC_ver2'

assert os.path.exists(PATH), PATH

dataset = datasets.load_from_disk(PATH)

In [3]:
from datasets import load_dataset
from tokenizers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

dataset = datasets.load_from_disk(PATH, streaming=True)

def encode(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

dataset = dataset.map(encode, batched=True)
next(iter(dataset))

ModuleNotFoundError: No module named 'tokenizers'

In [4]:
from collections.abc import Mapping
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union

In [5]:
InputDataClass = NewType("InputDataClass", Any)

"""
A DataCollator is a function that takes a list of samples from a Dataset and collate them into a batch, as a dictionary
of PyTorch/TensorFlow tensors or NumPy arrays.
"""

'\nA DataCollator is a function that takes a list of samples from a Dataset and collate them into a batch, as a dictionary\nof PyTorch/TensorFlow tensors or NumPy arrays.\n'

In [6]:
DataCollator = NewType("DataCollator", Callable[[List[InputDataClass]], Dict[str, Any]])

In [ ]:
class DataCollatorMixin:
    def __call__(self, features, return_tensors=None):
        if return_tensors is None:
            return_tensors = self.return_tensors
        if return_tensors == "tf":
            return self.tf_call(features)
        elif return_tensors == "pt":
            return self.torch_call(features)
        elif return_tensors == "np":
            return self.numpy_call(features)
        else:
            raise ValueError(f"Framework '{return_tensors}' not recognized!")


In [ ]:
def default_data_collator(features: List[InputDataClass], return_tensors="pt") -> Dict[str, Any]:
    """
    Very simple data collator that simply collates batches of dict-like objects and performs special handling for
    potential keys named:
        - `label`: handles a single value (int or float) per object
        - `label_ids`: handles a list of values per object
    Does not do any additional preprocessing: property names of the input object will be used as corresponding inputs
    to the model. See glue and ner for example of how it's useful.
    """

    # In this function we'll make the assumption that all `features` in the batch
    # have the same attributes.
    # So we will look at the first element as a proxy for what attributes exist
    # on the whole batch.

    if return_tensors == "pt":
        return torch_default_data_collator(features)


In [ ]:
@dataclass
class DefaultDataCollator(DataCollatorMixin):
    """
    Very simple data collator that simply collates batches of dict-like objects and performs special handling for
    potential keys named:
        - `label`: handles a single value (int or float) per object
        - `label_ids`: handles a list of values per object
    Does not do any additional preprocessing: property names of the input object will be used as corresponding inputs
    to the model. See glue and ner for example of how it's useful.
    This is an object (like other data collators) rather than a pure function like default_data_collator. This can be
    helpful if you need to set a return_tensors value at initialization.
    Args:
        return_tensors (`str`):
            The type of Tensor to return. Allowable values are "np", "pt" and "tf".
    """

    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Any]], return_tensors=None) -> Dict[str, Any]:
        if return_tensors is None:
            return_tensors = self.return_tensors
        return default_data_collator(features, return_tensors)


In [ ]:
def torch_default_data_collator(features: List[InputDataClass]) -> Dict[str, Any]:
    import torch

    if not isinstance(features[0], Mapping):
        features = [vars(f) for f in features]
    first = features[0]
    batch = {}

    # Special handling for labels.
    # Ensure that tensor is created with the correct type
    # (it should be automatically the case, but let's make sure of it.)
    if "label" in first and first["label"] is not None:
        label = first["label"].item() if isinstance(first["label"], torch.Tensor) else first["label"]
        dtype = torch.long if isinstance(label, int) else torch.float
        batch["labels"] = torch.tensor([f["label"] for f in features], dtype=dtype)
    elif "label_ids" in first and first["label_ids"] is not None:
        if isinstance(first["label_ids"], torch.Tensor):
            batch["labels"] = torch.stack([f["label_ids"] for f in features])
        else:
            dtype = torch.long if type(first["label_ids"][0]) is int else torch.float
            batch["labels"] = torch.tensor([f["label_ids"] for f in features], dtype=dtype)

    # Handling of all other possible keys.
    # Again, we will use the first element to figure out which key/values are not None for this model.
    for k, v in first.items():
        if k not in ("label", "label_ids") and v is not None and not isinstance(v, str):
            if isinstance(v, torch.Tensor):
                batch[k] = torch.stack([f[k] for f in features])
            elif isinstance(v, np.ndarray):
                batch[k] = torch.tensor(np.stack([f[k] for f in features]))
            else:
                batch[k] = torch.tensor([f[k] for f in features])

    return batch

# Streaming

```python
dataset = load_dataset("datasetname", split='train', streaming=True) -> IterableDataset
```
# An IterableDataset is useful for iterative jobs like training a model.

# Shuffling

```python
shuffled_dataset = dataset.shuffle(seed=42, buffer_size=10_000)
shuffled_dataset.set_epoch(epoch) # to (re)-shuffle the dataset at each epoch with a different seed
```



## data access

```python
IterableDataset.take(n) # returns the first n examples in a dataset:
IterableDataset.skip(n) # skips the first n examples in a dataset:
```

```python
# multilingual_dataset = interleave_datasets([en_dataset, fr_dataset]) # interleaves the examples of the datasets
from datasets import interleave_datasets
multilingual_dataset_with_oversampling = interleave_datasets([en_dataset, fr_dataset], probabilities=[0.8, 0.2], seed=42)

## Column manipulation
- IterableDataset.rename_column(old_name, new_name) # renames a column

## Filtering
```python
start_with_ar = dataset.filter(lambda example: example['text'].startswith('Ar'))
```



In [27]:
# !cat $PATH/dataset_info.json

In [28]:
humanize.number.intword(len(dataset))

'11.7 million'

In [29]:
doc = next(iter(dataset))

In [ ]:
# set the datasets cache
%env HF_DATASETS_CACHE = /fsx/mistobaan/datasets_cache

In [35]:
print(doc['text'])

Abstract
Whereas radio-diagnostic centers have potential to present hazardous effects due of ionizing radiation. Radio-diagnostic workers awareness, practices regarding radiation protection issues, availability of radiation protection devices and effective personal radiation exposure monitoring process has an important role to safe working in these places. We carried out this study in nine governmental Gaza governorates hospitals. The study instrument was close-ended structured questionnaire consists of five parts. 182 radio-diagnostic workers participated in the work. Based on the obtained data, the participants reported that 35.2% of personal radiation protection devices are available in the radio-diagnostic centers at governmental Gaza governorates hospitals. In spite the fact that 74.8% of participants have awareness about radiation protection issues, but it is only about 53.4% of participants follows the radiation protection practices. There is an obvious poor of personal radiatio

In [ ]:
# useuful snippet to load a dataset from a custom path
from datasets import load_dataset
dataset = load_dataset('LOADING_SCRIPT', cache_dir="PATH/TO/MY/CACHE/DIR")
from datasets import load_metric
metric = load_metric('glue', 'mrpc', cache_dir="MY/CACHE/DIRECTORY")
dataset.cleanup_cache_files()

# restart from scratch the transformation
updated_dataset = small_dataset.map(add_prefix, load_from_cache_file=False)

from datasets import disable_caching
disable_caching()

from datasets import load_metric
metric = load_metric('glue', 'mrpc', keep_in_memory=True)

Set `datasets.config.IN_MEMORY_MAX_SIZE` to a nonzero value (in bytes) that fits in your RAM memory.

Set the environment variable `HF_DATASETS_IN_MEMORY_MAX_SIZE` to a nonzero value. Note that the first method takes higher precedence.


# Tokenization

In [2]:
dataset.set_format(type="torch", 
                   columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
dataset.set_transform(transforms) # applies the transfomration on the fly
dataset = dataset.map(tokenization, batched=True)

NameError: name 'dataset' is not defined

In [7]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForMaskedLM, DataCollatorForLanguageModeling
from tqdm import tqdm
dataset = dataset.with_format("torch")
dataloader = DataLoader(dataset, collate_fn=DataCollatorForLanguageModeling(tokenizer))
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")
model.train().to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
for epoch in range(3):
    dataset.set_epoch(epoch)
    for i, batch in enumerate(tqdm(dataloader, total=5)):
        if i == 5:
            break
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

ModuleNotFoundError: No module named 'torch'